In [72]:
import pandas as pd
import numpy as np

In [112]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

In [73]:
data = pd.read_csv('./train.csv')

In [74]:
data.shape

(47102, 4)

In [75]:
data.head()

,id,name,document_text,cat_name
0,22474,Information Regarding the Merger of Navios Mar...,"At a special meeting held on March 24, 2021 sh...",Corporate Communications
1,27460,Announcement on Approving the Change of Member...,"On April 2, 2021, the China Financial Futures ...",Securities Settlement
2,6926,SFC Suspends Shiu Yau Wah for Five Months,The Securities and Futures Commission (SFC) ha...,Antitrust
3,6982,"Renminbi RMB Haircut - February 4, 2020",Pursuant to Section 2.6.2 of the Clearing Hous...,Securities Settlement
4,5022,"Anti-Money Laundering, Countering Financing of...",Money laundering and terrorism financing (ML/T...,Financial Crime


In [76]:
dummie = pd.get_dummies(data['cat_name'])

In [77]:
dummie

,Accounting and Finance,Antitrust,Banking,Broker Dealer,Commodities Trading,Compliance Management,Consumer protection,Contract Provisions,Corporate Communications,Corporate Governance,...,Required Disclosures,Research,Risk Management,Securities Clearing,Securities Issuing,Securities Management,Securities Sales,Securities Settlement,Trade Pricing,Trade Settlement
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47097,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
47098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47099,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
# data = data.reset_index(drop=True)
dummie = dummie.reset_index(drop=True)

In [79]:
dummie

,Accounting and Finance,Antitrust,Banking,Broker Dealer,Commodities Trading,Compliance Management,Consumer protection,Contract Provisions,Corporate Communications,Corporate Governance,...,Required Disclosures,Research,Risk Management,Securities Clearing,Securities Issuing,Securities Management,Securities Sales,Securities Settlement,Trade Pricing,Trade Settlement
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47097,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
47098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47099,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
data

,id,name,document_text,cat_name
0,22474,Information Regarding the Merger of Navios Mar...,"At a special meeting held on March 24, 2021 sh...",Corporate Communications
1,27460,Announcement on Approving the Change of Member...,"On April 2, 2021, the China Financial Futures ...",Securities Settlement
2,6926,SFC Suspends Shiu Yau Wah for Five Months,The Securities and Futures Commission (SFC) ha...,Antitrust
3,6982,"Renminbi RMB Haircut - February 4, 2020",Pursuant to Section 2.6.2 of the Clearing Hous...,Securities Settlement
4,5022,"Anti-Money Laundering, Countering Financing of...",Money laundering and terrorism financing (ML/T...,Financial Crime
...,...,...,...,...
47097,14825,Guide to the 2020 NSCC Fee Schedule,National Securities Clearing Corporation (NSCC...,Corporate Governance
47098,6875,Analytical Accounts of the Exchange Fund,"On January 14, 2020, the Hong Kong Monetary Au...",Monetary and Economic Policy
47099,8901,Adjudication Order In The Matter of Chiranjila...,Securities and Exchange Board of India ('SEBI'...,Fraud
47100,6285,Section 19 Letter for Criminal Conviction - Pr...,"The Federal Reserve Bank of Atlanta (""Reserve ...",Exemptions


In [81]:
data = pd.concat([data, dummie], axis=1)

In [82]:
data.head()

,id,name,document_text,cat_name,Accounting and Finance,Antitrust,Banking,Broker Dealer,Commodities Trading,Compliance Management,...,Required Disclosures,Research,Risk Management,Securities Clearing,Securities Issuing,Securities Management,Securities Sales,Securities Settlement,Trade Pricing,Trade Settlement
0,22474,Information Regarding the Merger of Navios Mar...,"At a special meeting held on March 24, 2021 sh...",Corporate Communications,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,27460,Announcement on Approving the Change of Member...,"On April 2, 2021, the China Financial Futures ...",Securities Settlement,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,6926,SFC Suspends Shiu Yau Wah for Five Months,The Securities and Futures Commission (SFC) ha...,Antitrust,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6982,"Renminbi RMB Haircut - February 4, 2020",Pursuant to Section 2.6.2 of the Clearing Hous...,Securities Settlement,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,5022,"Anti-Money Laundering, Countering Financing of...",Money laundering and terrorism financing (ML/T...,Financial Crime,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
data = data.drop(['id', 'cat_name'], axis=1)

In [85]:
data_doc = data.drop(['name'], axis=1)

In [86]:
data_doc.head()

,document_text,Accounting and Finance,Antitrust,Banking,Broker Dealer,Commodities Trading,Compliance Management,Consumer protection,Contract Provisions,Corporate Communications,...,Required Disclosures,Research,Risk Management,Securities Clearing,Securities Issuing,Securities Management,Securities Sales,Securities Settlement,Trade Pricing,Trade Settlement
0,"At a special meeting held on March 24, 2021 sh...",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,"On April 2, 2021, the China Financial Futures ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,The Securities and Futures Commission (SFC) ha...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Pursuant to Section 2.6.2 of the Clearing Hous...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Money laundering and terrorism financing (ML/T...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
from keras.preprocessing.text import Tokenizer

In [91]:
import re
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [92]:
labels = data_doc.drop(['document_text'], axis=1)

In [105]:
x = []
sentences = list(data_doc["document_text"])
for sen in sentences:
    x.append(preprocess_text(sen))

y = labels.values

In [106]:
len(y)

47102

In [107]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

In [108]:
x_train

['MBT Adjusted Mobile TeleSystems PJSC options will be subject to special pricing consideration in expiration processing on January The pricing consideration is due to the undetermined cash amount included in the option deliverable of MBT These options will continue to be subject to normal exercise based on customary exercise thresholds in OCC Ex by Ex processing New Deliverable per Contract will be Mobile TeleSystems PJSC MBT American Depositary Shares Approximately Cash less fees and withholdings if any As of January the MBT Distribution Agent has not yet determined the final net special dividend amount For purposes of calculating MBT price for use in expiration processing OCC will use the following formula MBT MBT ',
 'Authority for the Financial Markets AFM published that questions and answers become more relevant once they are more focused on the specific situation of the customer That is why it is important among other things that the questions are product and service specific In

In [109]:
from keras.preprocessing.sequence import pad_sequences

In [110]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [114]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('./glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [115]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(50, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [116]:
model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_split=0.2)

Epoch 1/5
251/251 [==============================] - 91s 358ms/step - loss: 0.1243 - acc: 0.0342 - val_loss: 0.0968 - val_acc: 0.0390
Epoch 2/5
251/251 [==============================] - 121s 484ms/step - loss: 0.0967 - acc: 0.0431 - val_loss: 0.0962 - val_acc: 0.0417
Epoch 3/5
251/251 [==============================] - 126s 503ms/step - loss: 0.0952 - acc: 0.0518 - val_loss: 0.0940 - val_acc: 0.0558
Epoch 4/5
251/251 [==============================] - 118s 470ms/step - loss: 0.0935 - acc: 0.0585 - val_loss: 0.0929 - val_acc: 0.0617
Epoch 5/5
251/251 [==============================] - 126s 501ms/step - loss: 0.0925 - acc: 0.0619 - val_loss: 0.0923 - val_acc: 0.0684


In [461]:
import tensorflow as tf
model = tf.keras.models.load_model('./model2.h5')

In [474]:
y_pred = model.predict(X_test)

In [475]:
prob = pd.DataFrame(y_pred)

In [476]:
prob.describe()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,7066.000000,7066.000000,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7066.000000,7.066000e+03,7066.000000,...,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03
mean,0.019038,0.014144,1.377175e-02,1.044137e-02,1.444584e-02,2.401872e-02,1.979938e-02,0.016270,6.928150e-03,0.018469,...,9.178231e-03,1.204895e-02,1.594837e-02,1.773228e-02,1.932244e-02,9.826208e-03,4.623758e-02,1.660891e-02,8.407463e-03,9.597874e-03
std,0.050740,0.032993,4.909920e-02,3.585567e-02,5.052989e-02,6.687929e-02,5.402656e-02,0.042445,2.589189e-02,0.042634,...,3.052941e-02,4.569004e-02,4.552556e-02,5.157080e-02,5.438925e-02,3.661485e-02,8.861612e-02,5.259460e-02,3.394054e-02,3.198647e-02
min,0.000003,0.000002,1.072778e-07,3.179380e-07,1.153115e-08,5.223188e-07,3.820152e-07,0.000002,4.304339e-07,0.000002,...,4.055205e-08,1.505725e-08,2.137963e-07,1.015105e-07,3.033746e-08,1.834376e-08,9.634750e-07,7.048742e-10,2.343080e-07,1.126167e-08
25%,0.000724,0.000328,1.432300e-04,2.125204e-04,6.863464e-05,4.556999e-04,5.901977e-04,0.000908,1.631901e-04,0.000908,...,2.412498e-04,7.979346e-05,3.281534e-04,1.548827e-04,3.474578e-04,2.050847e-04,3.178388e-03,1.177804e-04,1.101825e-04,9.874624e-05
50%,0.003464,0.002286,9.749681e-04,1.064330e-03,6.721318e-04,3.085315e-03,3.028050e-03,0.003455,7.082075e-04,0.004372,...,1.141712e-03,5.631894e-04,1.871616e-03,8.195937e-04,2.353221e-03,1.019612e-03,1.331002e-02,8.369982e-04,6.122142e-04,6.556958e-04
75%,0.014100,0.011662,6.076992e-03,4.996553e-03,5.789854e-03,1.456330e-02,1.467460e-02,0.011840,3.364839e-03,0.015707,...,5.407006e-03,4.149400e-03,9.177119e-03,6.092206e-03,1.317045e-02,5.076818e-03,4.628965e-02,6.732225e-03,3.415465e-03,3.836334e-03
max,0.712526,0.473585,6.158618e-01,5.594707e-01,7.334853e-01,8.180190e-01,7.619283e-01,0.738367,8.618800e-01,0.680069,...,4.706748e-01,8.091865e-01,7.074125e-01,5.397235e-01,9.267409e-01,6.564549e-01,8.903399e-01,7.103773e-01,7.254618e-01,5.778329e-01


In [477]:
y_pred[0]

array([2.7380735e-02, 1.6680360e-03, 9.9908173e-02, 2.9137731e-04,
       1.8137693e-04, 7.6103210e-04, 6.4158440e-04, 5.0715804e-03,
       2.1862984e-04, 4.8212826e-02, 3.0056834e-03, 8.6249113e-03,
       4.2358935e-03, 2.7813375e-02, 2.0756304e-02, 4.4913888e-03,
       1.0693073e-03, 3.7339330e-04, 3.2868981e-04, 1.8545985e-04,
       2.7839541e-03, 4.6116412e-03, 7.4333847e-03, 5.3955615e-03,
       1.8360913e-02, 3.4672022e-03, 1.1372209e-02, 1.2132525e-03,
       2.5075674e-04, 6.1732233e-03, 3.5455137e-02, 1.2261450e-02,
       9.4780028e-03, 1.8558502e-03, 1.9103289e-04, 3.5520196e-03,
       4.3380260e-04, 4.0484369e-03, 4.4907093e-02, 1.6604662e-03,
       3.1304359e-04, 1.2150943e-02, 4.7249198e-03, 1.8581748e-04,
       5.0356090e-03, 2.8622150e-04, 1.3719794e-01, 2.6801229e-04,
       6.4206986e-05, 7.4244046e-05], dtype=float32)

In [478]:
def man_f1(preds, value):
    for i in range(len(preds)):
        for j in range(len(preds[i])):
            if preds[i][j] > value:
                preds[i][j] = 1
            else:
                preds[i][j] = 0
    jodenahai = preds
    return jodenahai

In [479]:
y_pred

array([[2.73807347e-02, 1.66803598e-03, 9.99081731e-02, ...,
        2.68012285e-04, 6.42069863e-05, 7.42440461e-05],
       [2.04971433e-03, 1.60327554e-03, 5.67293167e-03, ...,
        1.50448084e-03, 4.51696217e-02, 4.24861908e-04],
       [3.64866853e-03, 5.74469566e-04, 8.34235549e-03, ...,
        1.98990107e-04, 1.53973699e-03, 6.11956129e-05],
       ...,
       [8.84529948e-03, 1.28984451e-04, 9.27828514e-05, ...,
        2.73674726e-04, 5.72382269e-05, 1.41569972e-03],
       [4.12046909e-04, 5.99384308e-04, 8.30582503e-05, ...,
        1.36494637e-04, 1.23501241e-01, 3.00991535e-03],
       [2.50285864e-03, 7.01895356e-03, 2.56538391e-04, ...,
        2.52932310e-04, 1.03233775e-04, 1.65790319e-04]], dtype=float32)

In [480]:
preds = pd.DataFrame(y_pred)

In [481]:
preds.describe()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,7066.000000,7066.000000,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7066.000000,7.066000e+03,7066.000000,...,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03,7.066000e+03
mean,0.019038,0.014144,1.377175e-02,1.044137e-02,1.444584e-02,2.401872e-02,1.979938e-02,0.016270,6.928150e-03,0.018469,...,9.178231e-03,1.204895e-02,1.594837e-02,1.773228e-02,1.932244e-02,9.826208e-03,4.623758e-02,1.660891e-02,8.407463e-03,9.597874e-03
std,0.050740,0.032993,4.909920e-02,3.585567e-02,5.052989e-02,6.687929e-02,5.402656e-02,0.042445,2.589189e-02,0.042634,...,3.052941e-02,4.569004e-02,4.552556e-02,5.157080e-02,5.438925e-02,3.661485e-02,8.861612e-02,5.259460e-02,3.394054e-02,3.198647e-02
min,0.000003,0.000002,1.072778e-07,3.179380e-07,1.153115e-08,5.223188e-07,3.820152e-07,0.000002,4.304339e-07,0.000002,...,4.055205e-08,1.505725e-08,2.137963e-07,1.015105e-07,3.033746e-08,1.834376e-08,9.634750e-07,7.048742e-10,2.343080e-07,1.126167e-08
25%,0.000724,0.000328,1.432300e-04,2.125204e-04,6.863464e-05,4.556999e-04,5.901977e-04,0.000908,1.631901e-04,0.000908,...,2.412498e-04,7.979346e-05,3.281534e-04,1.548827e-04,3.474578e-04,2.050847e-04,3.178388e-03,1.177804e-04,1.101825e-04,9.874624e-05
50%,0.003464,0.002286,9.749681e-04,1.064330e-03,6.721318e-04,3.085315e-03,3.028050e-03,0.003455,7.082075e-04,0.004372,...,1.141712e-03,5.631894e-04,1.871616e-03,8.195937e-04,2.353221e-03,1.019612e-03,1.331002e-02,8.369982e-04,6.122142e-04,6.556958e-04
75%,0.014100,0.011662,6.076992e-03,4.996553e-03,5.789854e-03,1.456330e-02,1.467460e-02,0.011840,3.364839e-03,0.015707,...,5.407006e-03,4.149400e-03,9.177119e-03,6.092206e-03,1.317045e-02,5.076818e-03,4.628965e-02,6.732225e-03,3.415465e-03,3.836334e-03
max,0.712526,0.473585,6.158618e-01,5.594707e-01,7.334853e-01,8.180190e-01,7.619283e-01,0.738367,8.618800e-01,0.680069,...,4.706748e-01,8.091865e-01,7.074125e-01,5.397235e-01,9.267409e-01,6.564549e-01,8.903399e-01,7.103773e-01,7.254618e-01,5.778329e-01


In [485]:
values = np.linspace(0.01, 0.04, 150)

In [486]:
values

array([0.01      , 0.01020134, 0.01040268, 0.01060403, 0.01080537,
       0.01100671, 0.01120805, 0.0114094 , 0.01161074, 0.01181208,
       0.01201342, 0.01221477, 0.01241611, 0.01261745, 0.01281879,
       0.01302013, 0.01322148, 0.01342282, 0.01362416, 0.0138255 ,
       0.01402685, 0.01422819, 0.01442953, 0.01463087, 0.01483221,
       0.01503356, 0.0152349 , 0.01543624, 0.01563758, 0.01583893,
       0.01604027, 0.01624161, 0.01644295, 0.0166443 , 0.01684564,
       0.01704698, 0.01724832, 0.01744966, 0.01765101, 0.01785235,
       0.01805369, 0.01825503, 0.01845638, 0.01865772, 0.01885906,
       0.0190604 , 0.01926174, 0.01946309, 0.01966443, 0.01986577,
       0.02006711, 0.02026846, 0.0204698 , 0.02067114, 0.02087248,
       0.02107383, 0.02127517, 0.02147651, 0.02167785, 0.02187919,
       0.02208054, 0.02228188, 0.02248322, 0.02268456, 0.02288591,
       0.02308725, 0.02328859, 0.02348993, 0.02369128, 0.02389262,
       0.02409396, 0.0242953 , 0.02449664, 0.02469799, 0.02489

In [487]:
from sklearn.metrics import f1_score
# values = [i for i in range(0, 1, 0.01)]
for i in values:
    y_pred = model.predict(X_test)
    print(i, f1_score(y_test, man_f1(y_pred, i), average='macro'))

0.01 0.06356716246752821
0.010201342281879194 0.06357480597426424
0.01040268456375839 0.0637018477444812
0.010604026845637583 0.06384403814601743
0.010805369127516779 0.06411077883629122
0.011006711409395973 0.06408694460306247
0.011208053691275168 0.06439886490498904
0.011409395973154362 0.06452946427936802
0.011610738255033557 0.06487516854613243
0.011812080536912751 0.06513019671518944
0.012013422818791947 0.06533710409360316
0.01221476510067114 0.06543987123271224
0.012416107382550336 0.06564134357595558
0.01261744966442953 0.06568468551077912
0.012818791946308725 0.0657272678882095
0.013020134228187919 0.06560715776404033
0.013221476510067115 0.06554616751437585
0.013422818791946308 0.06579648106106356
0.013624161073825504 0.06593450943560526
0.013825503355704698 0.06589557936658974
0.014026845637583892 0.0660958189082084
0.014228187919463087 0.06625767750865602
0.014429530201342283 0.06640643769487643
0.014630872483221476 0.06655498337730151
0.01483221476510067 0.0666561139932379

KeyboardInterrupt: 

In [488]:
man_f1(y_pred, 0.02812)

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [489]:
f1_score(y_test, man_f1(y_pred, 0.02812), average='macro')

0.07058033902982722

In [490]:
test = pd.DataFrame(y_test)

In [491]:
test.describe()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,7066.000000,7066.000000,7066.000000,7066.000000,7066.000000,7066.000000,7066.000000,7066.000000,7066.000000,7066.000000,...,7066.000000,7066.000000,7066.000000,7066.000000,7066.00000,7066.000000,7066.000000,7066.000000,7066.000000,7066.000000
mean,0.019672,0.019530,0.024483,0.014152,0.014011,0.028729,0.022078,0.027172,0.011463,0.020521,...,0.010756,0.012879,0.019247,0.026182,0.02321,0.015143,0.040900,0.017549,0.017266,0.014860
std,0.138879,0.138389,0.154555,0.118127,0.117543,0.167056,0.146946,0.162597,0.106459,0.141783,...,0.103158,0.112759,0.137402,0.159687,0.15058,0.122130,0.198073,0.131314,0.130269,0.121001
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


In [492]:
test = pd.read_csv('./test.csv')

In [493]:
test.head()

,id,name,document_text
0,4771,Companies (Amendment) Regulations 2020,Accounting and Corporate Regulatory Authority ...
1,4773,Notice of Intended Action Section 810-5-75-.31...,The Notice of intended action regarding sectio...
2,4787,Case and Desist Order in the Matter of Henry C...,"On January 9th, 2020, the Staff of the Arkansa..."
3,4791,AUSTRAC Further Strengthens International Part...,"On January 10, 2020, AUSTRAC and Great Britain..."
4,4794,Sydney Man Accused of Travelling to Philippine...,AUSTRAC published that a 63-year-old Sydney ma...


In [494]:
test = test.drop(['name'], axis=1)

In [495]:
test_list = []
sentences = list(test["document_text"])
for sen in sentences:
    test_list.append(preprocess_text(sen))

In [496]:
test_list

['Accounting and Corporate Regulatory Authority ACRA published that in exercise of the powers conferred by section of the Companies Act the Minister for Finance makes the Regulations The present Regulations may be cited as Companies Amendment Regulations and come into operation on January It specifically amends Regulation of the Companies Regulations Rg by deleting sub paragraphs h and and substituting the following sub paragraphs a person that has in force standard payment institution licence granted under section of the Payment Services Act Act of a person that has in force major payment institution licence granted or deemed to have been granted under section of the Payment Services Act an operator of payment system designated under section of the Payment Services Act and ia settlement institution of payment system designated under section of the Payment Services Act ',
 'The Notice of intended action regarding section Abandoned Motor Vehicle Record Request was published in the regis

In [497]:
test_token = tokenizer.texts_to_sequences(test_list)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

test = pad_sequences(test_token, padding='post', maxlen=maxlen)
# X_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [498]:
test

array([[ 893,    4,  374, ...,    0,    0,    0],
       [   1,   80,    2, ...,    0,    0,    0],
       [   7,  106,  991, ...,   32, 4607,    0],
       ...,
       [   7,   46,    1, ...,    0,    0,    0],
       [4129, 3464,   95, ...,    0,    0,    0],
       [   1,  518, 1032, ...,    0,    0,    0]])

In [499]:
x_train

array([[4618,  698, 2054, ...,    0,    0,    0],
       [ 137,    6,    1, ...,    0,    0,    0],
       [   7,  496,    1, ...,    0,    0,    0],
       ...,
       [   1,   23,   10, ...,   11,  216,  330],
       [ 324,    3,  257, ...,    0,    0,    0],
       [   1,  144,   17, ...,    0,    0,    0]])

In [500]:
test_predict = model.predict(test)

In [501]:
test_predict

array([[2.77736783e-03, 1.78715587e-03, 1.35927485e-06, ...,
        1.31087303e-02, 9.05568158e-05, 9.15551186e-03],
       [6.62502646e-03, 5.41597605e-04, 4.32512115e-06, ...,
        1.05240815e-05, 1.13684209e-05, 1.22649917e-05],
       [1.10856622e-01, 1.95026398e-04, 1.03663206e-02, ...,
        1.06089801e-05, 4.52709198e-03, 1.31249428e-04],
       ...,
       [4.00826633e-02, 6.45792484e-03, 4.78035212e-03, ...,
        3.01480293e-04, 8.78155231e-04, 6.97582960e-04],
       [1.41453147e-02, 1.87888443e-02, 3.82721424e-04, ...,
        2.50109188e-05, 1.13366608e-04, 6.77273874e-06],
       [2.01172829e-02, 1.90273523e-02, 7.23082125e-02, ...,
        3.55693519e-05, 3.59632795e-05, 2.16734024e-05]], dtype=float32)

In [502]:
for i in range(len(test_predict)):
    for j in range(len(test_predict[i])):
        if test_predict[i][j] > 0.028:
            test_predict[i][j] = 1
        else:
            test_predict[i][j] = 0

In [503]:
sub = pd.DataFrame(test_predict)

In [504]:
sub.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [505]:
test_csv = pd.read_csv('./test.csv')
sub['id'] = test_csv['id']

In [506]:
sub.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,id
0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4771
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4773
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4787
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4791
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4794


In [507]:
columns = dummie.columns

In [508]:
columns

Index(['Accounting and Finance', 'Antitrust', 'Banking', 'Broker Dealer',
       'Commodities Trading', 'Compliance Management', 'Consumer protection',
       'Contract Provisions', 'Corporate Communications',
       'Corporate Governance', 'Definitions', 'Delivery', 'Examinations',
       'Exemptions', 'Fees and Charges', 'Financial Accounting',
       'Financial Crime', 'Forms', 'Fraud', 'IT Risk', 'Information Filing',
       'Insurance', 'Legal', 'Legal Proceedings', 'Licensing',
       'Licensure and certification', 'Liquidity Risk', 'Listing',
       'Market Abuse', 'Market Risk', 'Monetary and Economic Policy',
       'Money Services', 'Money-Laundering and Terrorist Financing',
       'Natural Disasters', 'Payments and Settlements', 'Powers and Duties',
       'Quotation', 'Records Maintenance', 'Regulatory Actions',
       'Regulatory Reporting', 'Required Disclosures', 'Research',
       'Risk Management', 'Securities Clearing', 'Securities Issuing',
       'Securities Manage

In [509]:
columns[0]

'Accounting and Finance'

In [510]:
submission = []

In [511]:
sub.iat[i, 50]

57235

In [512]:
submission = []

In [513]:
for i in range(len(sub)):
    for j in range(50):
        submission.append([f'{sub.iat[i, 50]}_{columns[j]}', int(sub.iat[i, j])])

In [514]:
submission

[['4771_Accounting and Finance', 0],
 ['4771_Antitrust', 0],
 ['4771_Banking', 0],
 ['4771_Broker Dealer', 1],
 ['4771_Commodities Trading', 0],
 ['4771_Compliance Management', 1],
 ['4771_Consumer protection', 1],
 ['4771_Contract Provisions', 1],
 ['4771_Corporate Communications', 0],
 ['4771_Corporate Governance', 0],
 ['4771_Definitions', 0],
 ['4771_Delivery', 0],
 ['4771_Examinations', 0],
 ['4771_Exemptions', 1],
 ['4771_Fees and Charges', 1],
 ['4771_Financial Accounting', 0],
 ['4771_Financial Crime', 0],
 ['4771_Forms', 0],
 ['4771_Fraud', 0],
 ['4771_IT Risk', 0],
 ['4771_Information Filing', 0],
 ['4771_Insurance', 0],
 ['4771_Legal', 0],
 ['4771_Legal Proceedings', 0],
 ['4771_Licensing', 0],
 ['4771_Licensure and certification', 1],
 ['4771_Liquidity Risk', 0],
 ['4771_Listing', 0],
 ['4771_Market Abuse', 0],
 ['4771_Market Risk', 0],
 ['4771_Monetary and Economic Policy', 0],
 ['4771_Money Services', 0],
 ['4771_Money-Laundering and Terrorist Financing', 0],
 ['4771_Natu

In [515]:
sample_sub = pd.DataFrame(submission)

In [516]:
sample_sub.columns = ['id', 'predictions']

In [517]:
sample_sub.head()

,id,predictions
0,4771_Accounting and Finance,0
1,4771_Antitrust,0
2,4771_Banking,0
3,4771_Broker Dealer,1
4,4771_Commodities Trading,0


In [518]:
sample_sub.to_csv('./fifth.csv', index=False)

In [1]:
train = pd.read_csv('./train.csv')

NameError: name 'pd' is not defined